In [47]:
import pandas as pd
import sqlite3
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',100)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()
games_2021 = pd.read_sql_query("SELECT * FROM [2020-2021 Team Data Combined]", conn)
games_2022 = pd.read_sql_query("SELECT * FROM [2021-2022 Team Data Combined]", conn)
games_2023 = pd.read_sql_query("SELECT * FROM [2022-2023 Team Data Combined]", conn)
games_2024 = pd.read_sql_query("SELECT * FROM [2023-2024 Team Data Combined]", conn)
games_2025 = pd.read_sql_query("SELECT * FROM [2024-2025 Team Data Combined]", conn)
lst_of_szns = [games_2021, games_2022, games_2023, games_2024, games_2025]
for x in lst_of_szns:
    x['GAME_DATE'] = pd.to_datetime(x['GAME_DATE'])

In [48]:
betting = pd.read_csv("nba_2008-2025.csv")
szn = betting["season"] > 2020
regular_szn = betting['regular'] == True
betting = betting[szn]
betting = betting[regular_szn]
betting

/var/folders/j1/x2dgrv6501l9gqdy9k6_31wc0000gn/T/ipykernel_10358/3686932895.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  betting = betting[regular_szn]


,season,date,regular,playoffs,away,home,score_away,score_home,q1_away,q2_away,q3_away,q4_away,ot_away,q1_home,q2_home,q3_home,q4_home,ot_home,whos_favored,spread,total,moneyline_away,moneyline_home,h2_spread,h2_total,id_spread,id_total
16664,2021,2020-12-22,True,False,gs,bkn,99,125,25,20,26,28,0,40,23,36,26,0,home,7.5,234.5,265.0,-320.0,1.0,115.0,1.0,0
16665,2021,2020-12-22,True,False,lac,lal,116,109,39,17,33,27,0,19,35,24,31,0,home,2.0,219.0,115.0,-135.0,3.5,108.5,0.0,1
16666,2021,2020-12-23,True,False,cha,cle,114,121,23,21,36,34,0,24,41,28,28,0,away,3.0,217.0,-140.0,120.0,7.0,107.0,0.0,1
16667,2021,2020-12-23,True,False,ny,ind,107,121,33,33,16,25,0,35,26,27,33,0,home,7.5,215.5,270.0,-330.0,7.5,107.5,1.0,1
16668,2021,2020-12-23,True,False,mia,orl,107,113,29,24,30,24,0,27,29,23,34,0,away,4.5,219.0,-190.0,170.0,5.0,106.5,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23023,2025,2025-04-13,True,False,cha,bos,86,93,15,28,25,18,0,33,21,15,24,0,home,10.5,208.5,NaN,NaN,NaN,NaN,0.0,0
23024,2025,2025-04-13,True,False,wsh,mia,119,118,30,36,28,25,0,28,32,26,32,0,home,8.5,215.5,NaN,NaN,NaN,NaN,0.0,1
23025,2025,2025-04-13,True,False,ny,bkn,113,105,38,24,27,24,0,37,29,17,22,0,away,5.5,209.5,NaN,NaN,NaN,NaN,1.0,1
23026,2025,2025-04-13,True,False,det,mil,133,140,25,30,37,31,10,43,28,27,25,17,away,9.5,222.5,NaN,NaN,NaN,NaN,0.0,1


In [49]:
betting.drop(['regular', 'q1_away', 'q2_away', 'q3_away', 'q4_away', 'ot_away', 'q1_home', 'q2_home', 'q3_home','q4_home', 'ot_home'],axis=1, inplace=True)
betting['date'] = pd.to_datetime(betting['date'])

In [50]:
team_mapping = {
    'gs': (1610612744, 'GSW'),
    'lac': (1610612746, 'LAC'),
    'cha': (1610612766, 'CHA'),
    'ny': (1610612752, 'NYK'),
    'mia': (1610612748, 'MIA'),
    'wsh': (1610612764, 'WAS'),
    'mil': (1610612749, 'MIL'),
    'no': (1610612740, 'NOP'),
    'atl': (1610612737, 'ATL'),
    'sa': (1610612759, 'SAS'),
    'det': (1610612765, 'DET'),
    'sac': (1610612758, 'SAC'),
    'utah': (1610612762, 'UTA'),
    'dal': (1610612742, 'DAL'),
    'bkn': (1610612751, 'BKN'),
    'cle': (1610612739, 'CLE'),
    'orl': (1610612753, 'ORL'),
    'okc': (1610612760, 'OKC'),
    'phi': (1610612755, 'PHI'),
    'ind': (1610612754, 'IND'),
    'tor': (1610612761, 'TOR'),
    'min': (1610612750, 'MIN'),
    'phx': (1610612756, 'PHX'),
    'hou': (1610612745, 'HOU'),
    'bos': (1610612738, 'BOS'),
    'mem': (1610612763, 'MEM'),
    'por': (1610612757, 'POR'),
    'chi': (1610612741, 'CHI'),
    'den': (1610612743, 'DEN'),
    'lal': (1610612747, 'LAL')
}

In [51]:
def away_TEAM_ID(x):
    return team_mapping[x['away']][0]
def away_TEAM_NAME(x):
    return team_mapping[x['away']][1]
def home_TEAM_ID(x):
    return team_mapping[x['home']][0]
def home_TEAM_NAME(x):
    return team_mapping[x['home']][1]
def home_SPREAD(x):
    if (x['whos_favored'] == 'home'):
        return x['spread'] * -1
    return x['spread'] 
def away_SPREAD(x):
    if (x['whos_favored'] == 'away'):
        return x['spread'] * - 1
    return x['spread']

In [52]:
betting['away_TEAM_ID'] = betting.apply(away_TEAM_ID, axis=1)
betting['away_TEAM_NAME'] = betting.apply(away_TEAM_NAME, axis=1)
betting['home_TEAM_NAME'] = betting.apply(home_TEAM_NAME, axis=1)
betting['home_TEAM_ID'] = betting.apply(home_TEAM_ID, axis=1)
betting['home_SPREAD'] = betting.apply(home_SPREAD, axis=1)
betting['away_SPREAD'] = betting.apply(away_SPREAD, axis=1)

In [53]:
betting.drop(['playoffs', 'away', 'home', 'score_home', 'score_away', 'h2_spread','h2_total','whos_favored', 'spread'], axis=1,inplace=True)

In [54]:
dfs = {}
szns = [2021,2022,2023,2024,2025]

In [55]:
for szn in szns:
    szn_df = betting[betting['season'] == szn].copy()
    dfs[szn] = szn_df
szn_2021 = dfs[2021]
szn_2022 = dfs[2022]
szn_2023 = dfs[2023]
szn_2024 = dfs[2024]
szn_2025 = dfs[2025]

In [56]:
szn_2021_merge = pd.merge(games_2021, szn_2021, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])
szn_2022_merge = pd.merge(games_2022, szn_2022, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])
szn_2023_merge = pd.merge(games_2023, szn_2023, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])
szn_2024_merge = pd.merge(games_2024, szn_2024, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])
szn_2025_merge = pd.merge(games_2025, szn_2025, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])

In [57]:
szn_2021_merge

,GAME_ID,GAME_DATE,SEASON_ID,home_TEAM_ID,home_TEAM,home_TEAM_NAME_x,MATCHUP_x,home_WL,home_MIN,home_PTS,home_FGM,home_FGA,home_FG_PCT,home_FG3M,home_FG3A,home_FG3_PCT,home_FTM,home_FTA,home_FT_PCT,home_OREB,home_DREB,home_REB,home_AST,home_STL,home_BLK,home_TOV,home_PF,home_PLUS_MINUS,home_TotalGamePoints,home_TotalGameFTM,home_TotalDREB,home_TotalAST,home_TotalSTL,home_TotalBLK,home_TotalPF,home_EFG,home_TOVP,home_OREBP,home_FTRATE,home_OppEFG,home_OppTOVP,home_DREBP,home_OppFTRATE,home_TEAMPIE,home_EWMA PTS,home_EWMA FGA,home_EWMA FG_PCT,home_EWMA FG3M,home_EWMA FG3A,home_EWMA FG3_PCT,home_EWMA FTM,home_EWMA FTA,home_EWMA FT_PCT,home_EWMA OREB,home_EWMA DREB,home_EWMA AST,home_EWMA STL,home_EWMA TOV,home_EWMA PF,home_EWMA PLUS_MINUS,home_EWMA EFG,home_EWMA TOVP,home_EWMA OREBP,home_EWMA FTRATE,home_EWMA OppEFG,home_EWMA OppTOVP,home_EWMA DREBP,home_EWMA OppFTRATE,home_EWMA TEAMPIE,away_TEAM_ID,away_TEAM,away_TEAM_NAME_x,MATCHUP_y,away_WL,away_MIN,away_PTS,away_FGM,away_FGA,away_FG_PCT,away_FG3M,away_FG3A,away_FG3_PCT,away_FTM,away_FTA,away_FT_PCT,away_OREB,away_DREB,away_REB,away_AST,away_STL,away_BLK,away_TOV,away_PF,away_PLUS_MINUS,away_TotalGamePoints,away_TotalGameFTM,away_TotalDREB,away_TotalAST,away_TotalSTL,away_TotalBLK,away_TotalPF,away_EFG,away_TOVP,away_OREBP,away_FTRATE,away_OppEFG,away_OppTOVP,away_DREBP,away_OppFTRATE,away_TEAMPIE,away_EWMA PTS,away_EWMA FGA,away_EWMA FG_PCT,away_EWMA FG3M,away_EWMA FG3A,away_EWMA FG3_PCT,away_EWMA FTM,away_EWMA FTA,away_EWMA FT_PCT,away_EWMA OREB,away_EWMA DREB,away_EWMA AST,away_EWMA STL,away_EWMA TOV,away_EWMA PF,away_EWMA PLUS_MINUS,away_EWMA EFG,away_EWMA TOVP,away_EWMA OREBP,away_EWMA FTRATE,away_EWMA OppEFG,away_EWMA OppTOVP,away_EWMA DREBP,away_EWMA OppFTRATE,away_EWMA TEAMPIE,EWMA PTS DIFF,EWMA FGA DIFF,EWMA FG_PCT DIFF,EWMA FG3M DIFF,EWMA FG3A DIFF,EWMA FG3_PCT DIFF,EWMA FTM DIFF,EWMA FTA DIFF,EWMA FT_PCT DIFF,EWMA OREB DIFF,EWMA DREB DIFF,EWMA AST DIFF,EWMA STL DIFF,EWMA TOV DIFF,EWMA PF DIFF,EWMA PLUS_MINUS DIFF,EWMA EFG DIFF,EWMA TOVP DIFF,EWMA OREBP DIFF,EWMA FTRATE DIFF,EWMA OppEFG DIFF,EWMA OppTOVP DIFF,EWMA DREBP DIFF,EWMA OppFTRATE DIFF,EWMA TEAMPIE DIFF,home_win,DIFF,home_ELO,away_ELO,away_POSS,home_POSS,away_PACE,home_PACE,home_OffRating,away_OffRating,home_DefRating,away_DefRating,home_NetRating,away_NetRating,home REST DAYS,away REST DAYS,home Back To Back,away Back To Back,elo DIFF,rest DIFF,season,date,total,moneyline_away,moneyline_home,id_spread,id_total,away_TEAM_NAME_y,home_TEAM_NAME_y,home_SPREAD,away_SPREAD
0,0022000002,2020-12-22,22020,1610612747,LAL,Los Angeles Lakers,LAL vs. LAC,L,240,109,38,81,0.469,9,29,0.310,24,31,0.774,8,37,45,22,4,2,19,20,-7.0,225,38,66,44,14,5,49,0.525,0.1672,0.216216,0.382716,0.548,0.136,0.770833,0.204301,0.5087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612746,LAC,LA Clippers,LAC @ LAL,W,241,116,44,93,0.473,14,40,0.350,14,19,0.737,11,29,40,22,10,3,16,29,7.0,225,38,66,44,14,5,49,0.548,0.1363,0.229167,0.204301,0.525,0.167,0.783784,0.382716,0.4913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7,1500.000000,1500.000000,106.36,105.64,105.918672,105.640000,103.180613,109.063558,109.063558,103.180613,-5.882944,5.882944,NaN,NaN,0,0,0.000000,NaN,2021,2020-12-22,219.0,115.0,-135.0,0.0,1,LAC,LAL,-2.0,2.0
1,0022000001,2020-12-22,22020,1610612751,BKN,Brooklyn Nets,BKN vs. GSW,W,242,125,42,92,0.457,15,35,0.429,26,32,0.813,13,44,57,24,11,7,20,22,26.0,224,41,78,50,17,13,46,0.538,0.1586,0.276596,0.347826,0.424,0.142,0.771930,0.232323,0.6499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612744,GSW,Golden State Warriors,GSW @ BKN,L,240,99,37,99,0.374,10,33,0.303,15,23,0.652,13,34,47,26,6,6,18,24,-26.0,224,41,78,50,17,13,46,0.424,0.1416,0.228070,0.232323,0.538,0.159,0.7

In [58]:
szn_2021_merge.to_sql("2020-2021 Team Data Combined", conn, if_exists="replace", index=False)
szn_2022_merge.to_sql("2021-2022 Team Data Combined", conn, if_exists="replace", index=False)
szn_2023_merge.to_sql("2022-2023 Team Data Combined", conn, if_exists="replace", index=False)
szn_2024_merge.to_sql("2023-2024 Team Data Combined", conn, if_exists="replace", index=False)
szn_2025_merge.to_sql("2024-2025 Team Data Combined", conn, if_exists="replace", index=False)

1225